# Preparation of TReX dataset

In [1]:
# build a rdf dictionary from wikidata truthy
import json

entity_dictionary = {}
with open("/data/wikidata/latest-truthy.nt", "r") as f:
    for line in f:
        if "rdf-schema#label" in line and "@en " in line:
            try:
                s,p ,o = line.split("> ")
                s = s.replace("<", "")
                o = o.replace("@en", "")
                start = o.index( "\"" ) + 1
                end = o.index( "\"", start )
                o = o[start:end]
                s = s.encode('utf-8').decode('unicode-escape')
                o = o.encode('utf-8').decode('unicode-escape')
                    
                s = s.replace("http://www.wikidata.org/entity/","")
                o = o.replace("http://www.wikidata.org/entity/","")
                
                entity_dictionary[s] = o
            except ValueError:
                continue
#with open('data/data/entity_dictionary', 'w') as fp:
#    json.dump(entity_dictionary, fp, indent=4, sort_keys=True)
        

In [ ]:
print("test")

In [2]:
#Helper

def remove_duplicates(triples):
    seen = set()
    new_l = []
    for d in triples:
        t = tuple(d.items())
        if t not in seen:
            seen.add(t)
            new_l.append(d)
    return new_l

def substract_triples(original_triples, substract_triples):
    
    to_be_removed = set()
    for d in substract_triples:
        to_be_removed.add(tuple(d.items()))
    
    new_l = []
    for d in original_triples:
        t = tuple(d.items())
        if t not in to_be_removed:
            new_l.append(d)
    return new_l

In [4]:
import json, os
from transformers import BertTokenizer



trex_folder = "data/data/TREx/"
entity_dictionary_file = "data/data/entity_dictionary"

common_vocab_file = "./common_vocab_cased.txt"

trex_triple_file = "data/"+"trex_test.json"
output_file = []







def get_label(uri,entity_dictionary):
    return entity_dictionary[uri] 

def load_trex_file(trex_data, entity_dictionary, common_vocab):
    global output_file
    counter = 0
    for triple_dict in trex_data:
        #print(triple_dict)
        s_qid = triple_dict["sub_uri"]
        p_qid = triple_dict["predicate_id"]
        o_qid = triple_dict["obj_uri"]
        s_label = triple_dict["sub_label"]
        o_label = triple_dict["obj_label"]
        try:
            triple = {"subject": s_label, "predicate": p_qid, "object": o_label}
            if o_label in common_vocab:
                counter += 1
                output_file.append(triple)
        except KeyError:
            continue
    print("Added {} triples of property {}.".format(counter, p_qid))
    





#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

with open(entity_dictionary_file, "r") as f:
    entity_dictionary = json.load(f)
    with open(common_vocab_file, "r") as f:
        common_vocab = set()
        for line in f:
            common_vocab.add(line.strip())
        for file in os.listdir(trex_folder):
            with open(trex_folder+file, "r") as f:
                trex_data = []
                for line in f.readlines():
                    trex_data.append(json.loads(line))
                load_trex_file(trex_data, entity_dictionary, common_vocab)

        print("Lengths before removal {}.".format(len(output_file)))
        output_file = remove_duplicates(output_file)
        print("Lengths after removal {}.".format(len(output_file)))
        with open(trex_triple_file, "w") as f:
            json.dump(output_file,f, indent=4, sort_keys=True)
#print(output_file)
    


Added 976 triples of property P527.
Added 977 triples of property P103.
Added 964 triples of property P279.
Added 936 triples of property P740.
Added 949 triples of property P1303.
Added 881 triples of property P449.
Added 473 triples of property P140.
Added 687 triples of property P127.
Added 953 triples of property P20.
Added 975 triples of property P30.
Added 922 triples of property P31.
Added 645 triples of property P138.
Added 954 triples of property P937.
Added 995 triples of property P190.
Added 856 triples of property P364.
Added 701 triples of property P1001.
Added 892 triples of property P39.
Added 958 triples of property P106.
Added 234 triples of property P1376.
Added 703 triples of property P36.
Added 952 triples of property P413.
Added 881 triples of property P131.
Added 966 triples of property P27.
Added 877 triples of property P407.
Added 592 triples of property P178.
Added 696 triples of property P101.
Added 969 triples of property P1412.
Added 922 triples of property 

NameError: name 'triple_dict' is not defined

# get Wikidata training data



In [6]:
training_data =[]
properties = {"P1001", "P106", "P1303", "P1376", "P1412", "P178", "P19", "P276", "P30", "P364", "P39", "P449", "P495", "P740", "P101", "P108", "P131", "P138", "P159", "P17", "P20", "P279", "P31", "P36", "P407", "P463", "P527", "P937", "P103", "P127", "P136", "P140", "P176", "P190", "P264", "P27", "P361", "P37", "P413", "P47", "P530"}
wikidata_training_path = "data/data/TREx/wikidata_training.json"



def get_label(uri,entity_dictionary):
    return entity_dictionary[uri] 






with open(entity_dictionary_file, "r") as f:
    entity_dictionary = json.load(f)
    with open(common_vocab_file, "r") as f:
        common_vocab = set()
        for line in f:
            common_vocab.add(line.strip())
        with open("/data/wikidata/latest-truthy.nt", "r") as f:
            for line in f:
                try:
                    s,p,o = line.split("> <")
                    if "wikidata.org/entity" in o and "wikidata.org/entity" in s:

                        s = s.replace("<", "")
                        o = o.replace("> .\n","")
                        s = s.replace("http://www.wikidata.org/entity/","")
                        o = o.replace("http://www.wikidata.org/entity/","")
                        p = p.replace("http://www.wikidata.org/prop/direct/","")
                        p = p.replace(">","")
                        if p in properties:
                            try:
                                s = get_label(s, entity_dictionary)
                                o = get_label(o, entity_dictionary)
                            except KeyError:
                                continue
                            if o in common_vocab:
                                triple = {"subject": s, "predicate": p, "object": o}
                                training_data.append(triple)
                except ValueError:
                    continue

#remove testdata from training data

training_data = remove_duplicates(training_data)
training_data = substract_triples(training_data, output_file)

#with open(wikidata_training_path, "w") as f:
#    json.dump(training_data, f, indent=4, sort_keys=True)

In [60]:
def check_triple(obj_label, tokenizer):
    tokenized_obj = tokenizer.tokenize(obj_label)
    if "[UNK]" not in tokenized_obj:
        if len(tokenized_obj) == 1:
            return True
    return False

with open("/data/kalo/iswc2020/LAMA/common_vocab_cased.txt", "r") as f:
    for line in f:
        if check_triple(line, tokenizer) == False:
            print(line)
        else:
            print(line)
            break


#



In [10]:
# counter for properties in wikidata training dataset
from collections import Counter
c = []
with open(wikidata_training_path, "r") as f:
    training_data = json.load(f)
    for d in training_data:
        c.append(d["predicate"])
        
counter = Counter(c)
print(counter.most_common)

<bound method Counter.most_common of Counter({'P31': 20508383, 'P407': 11317828, 'P17': 5201482, 'P106': 5053504, 'P27': 2221662, 'P1412': 1782630, 'P279': 1077521, 'P19': 828824, 'P131': 780222, 'P20': 412882, 'P495': 347826, 'P413': 312834, 'P136': 231264, 'P364': 228731, 'P937': 175686, 'P1303': 163219, 'P1001': 149718, 'P103': 148823, 'P276': 121847, 'P159': 114081, 'P140': 98585, 'P39': 58426, 'P101': 50890, 'P30': 49295, 'P527': 44526, 'P138': 19536, 'P361': 19473, 'P740': 18941, 'P264': 14681, 'P449': 10660, 'P37': 10621, 'P47': 7392, 'P176': 6186, 'P108': 6041, 'P127': 4884, 'P190': 4749, 'P530': 4195, 'P178': 3223, 'P36': 2500, 'P463': 1757, 'P1376': 337})>
